# Init

In [1]:
# # if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget
# !pip install gensim
# !pip install catboost
# !pip install cython

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
import sys
import logging
import gc
import wget
import time
import tarfile
import zipfile
import functools
import random
import copy
import pickle
from tqdm import tqdm_notebook, tqdm
from itertools import product, combinations
from scipy.special import comb, perm

import scipy
import numpy as np
import pandas as pd
import catboost as cbt
from glove import *

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.linear_model import LogisticRegression,BayesianRidge,SGDClassifier,PassiveAggressiveClassifier,RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC,NuSVC,SVC
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers

# import keras
# import keras.backend as K
# from keras import layers


import gensim
import xgboost as xgb
import lightgbm as lgb

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
# import ray
# ray.init(object_store_memory=int(100e6))
# import modin.pandas as pd
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

In [5]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a')
        
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        
    def flush(self):
        pass

In [6]:
# sys.stdout = Logger("logs/default.log", sys.stdout)
# sys.stderr = Logger("logs/default_err.log", sys.stderr)

In [7]:
DDIR = "data"
UDDIR = "user_data"
UFEDIR = "user_data/feat_data_v05"
UMDIR = "user_data/model_data"
RESDIR = "prediction_result"

In [8]:
UID = "user_id"

# Load data (Only once)

In [9]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [10]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [11]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [12]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

In [13]:
def mynunique(values):
    return values.nunique(dropna=False)
def getidxmax(x):
    return x.idxmax()[1]
# for time series
def at_len(x):
    return len(x)

def at_sum(x):
    return np.sum(x)

def at_max(x):
    return np.max(x)

def at_min(x):
    return np.min(x)

def at_mean(x):
    return np.mean(x)

def at_range(x):
    return at_max(x) - at_min(x)

def at_nunq(x):
    return len(set(x))

def at_lenDrange(x):
    return at_len(x)/(at_range(x)+1)

def at_lenDnunq(x):
    return at_len(x)/at_nunq(x)

def at_percentile(n):
    def at_percentile_(x):
        return np.percentile(x, n)
    at_percentile_.__name__ = f"at_percentile_{n}"
    return at_percentile_

In [14]:
OP_SET = ["sum", "max", "min", "mean", "std"]
OP_SET1 = ["nunique", "sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]
OP_SET2 = ["sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]

In [15]:
ID_SET = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry", "time", "click_times"]

In [16]:
nesting_level = 0
is_start = None
class Timer:
    def __init__(self):
        self.start = time.time()
        self.history = [self.start]

    def check(self, info):
        current = time.time()
        print(f"[{info}] spend {current - self.history[-1]:0.2f} sec")
        self.history.append(current)

def log(entry):
    global nesting_level
    space = "-" * (4 * nesting_level)
    print(f"{space}{entry}")

def timeit(method, start_log=None):
    @functools.wraps(method)
    def timed(*args, **kw):
        global is_start
        global nesting_level

        if not is_start:
            print()

        is_start = True
        log(f"Start [{method.__name__}]:" + (start_log if start_log else ""))
        log(f'Start time: {time.strftime("%Y-%m-%d %H:%M:%S")}')
        nesting_level += 1

        start_time = time.time()
        result = method(*args, **kw)
        end_time = time.time()

        nesting_level -= 1
        log(f"End   [{method.__name__}]. Time elapsed: {end_time - start_time:0.2f} sec.")
        is_start = False

        return result

    return timed

In [17]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Preprocessing

In [18]:
feat_fname = sorted(os.listdir(UFEDIR))

In [19]:
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

## Padding Sequence

In [20]:
def loadnprec(col, max_features=None, maxlen=None, train_order=train_user[UID].values, test_order=test_user[UID].values):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    
    # train_seq[col] = train_seq[col].apply(lambda x: " ".join(x))
    # test_seq[col] = test_seq[col].apply(lambda x: " ".join(x))
    
    train_seq.sort_index(inplace=True)
    test_seq.sort_index(inplace=True)
    
    assert (train_seq.index.values != train_order).sum() == 0
    assert (test_seq.index.values != test_order).sum() == 0
    
    train_X = train_seq[col].values
    test_X = test_seq[col].values
    
    tol_X = np.concatenate([train_X, test_X])
    
    # Tokenize the sentences
    tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(tol_X))
    
    tol_X = tokenizer.texts_to_sequences(tol_X)

    # Pad the sentences 
    tol_X = keras.preprocessing.sequence.pad_sequences(tol_X, maxlen=maxlen, padding="post")
    
    train_X = tol_X[:len(train_seq)]
    test_X = tol_X[len(train_seq):]
    
    return train_X, test_X, tokenizer.word_index

In [21]:
def load_glove(col, word_index, max_features):
    
    EMBEDDING_FILE = (f"{UMDIR}/vectors/glove_{col}.model")
    k = Glove.load(EMBEDDING_FILE)
    
    embeddings_index = []
    for i in tqdm(k.dictionary.keys()):
        embeddings_index.append((i,k.word_vectors[k.dictionary[i]]))

    embeddings_index = dict(pd.DataFrame(embeddings_index).values)
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    
    for word, i in word_index.items():
        if i >= nb_words: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 

In [22]:
def load_w2v(col, word_index, max_features, embed_size):    
    EMBEDDING_FILE = (f"{UMDIR}/vectors/w2v{embed_size}/w2v_{col}.model")
    
    model = gensim.models.Word2Vec.load(EMBEDDING_FILE)
    assert list(word_index.keys()) == model.wv.index2word
    
    embeddings_index = dict()
    for word in model.wv.index2word:
        embeddings_index[word] = model.wv[word]
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    if max_features is None:
        nb_words = len(word_index)+1
    else:
        nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= nb_words: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [23]:
USE_IDS = ["creative_id", "ad_id", "advertiser_id",]   # "product_id", "product_category", "industry", 

In [24]:
# default max_features 5000
maxlen = 128
embed_size = 128
max_features = None
# max_features = 50000

In [29]:
train_textprec_dic = dict()
test_textprec_dic = dict()
w2v_wordidx_dic = dict()
emb_matrix_dic = dict()

In [30]:
np.random.seed(2020)
for cur_id in tqdm(USE_IDS):
    train_textprec_dic[cur_id], test_textprec_dic[cur_id], w2v_wordidx_dic[cur_id] = loadnprec(cur_id, max_features, maxlen)
    emb_matrix_dic[cur_id] = load_w2v(cur_id, w2v_wordidx_dic[cur_id], max_features, embed_size)

100%|██████████| 3/3 [07:22<00:00, 147.58s/it]


In [31]:
with open(f"{UDDIR}/imd/train_textprec_dic_w2v{embed_size}_seq{maxlen}_mf{max_features}.pkl", "wb") as f:
    pickle.dump(pickle.dumps(train_textprec_dic), f, protocol=4)

with open(f"{UDDIR}/imd/test_textprec_dic_w2v{embed_size}_seq{maxlen}_mf{max_features}.pkl", "wb") as f:
    pickle.dump(pickle.dumps(test_textprec_dic), f, protocol=4)

with open(f"{UDDIR}/imd/w2v_wordidx_dic_w2v{embed_size}_seq{maxlen}_mf{max_features}.pkl", "wb") as f:
    pickle.dump(pickle.dumps(w2v_wordidx_dic), f, protocol=4)

with open(f"{UDDIR}/imd/emb_matrix_dic_w2v{embed_size}_seq{maxlen}_mf{max_features}.pkl", "wb") as f:
    pickle.dump(pickle.dumps(emb_matrix_dic), f, protocol=4)

In [32]:
# with open(f"{UDDIR}/imd/train_textprec_dic_w2v{embed_size}_seq{maxlen}_mf{max_features}.pkl", "rb") as f:
#     train_textprec_dic = pickle.loads(pickle.load(f))
    
# with open(f"{UDDIR}/imd/test_textprec_dic_w2v{embed_size}_seq{maxlen}_mf{max_features}.pkl", "rb") as f:
#     test_textprec_dic = pickle.loads(pickle.load(f))

# with open(f"{UDDIR}/imd/w2v_wordidx_dic_w2v{embed_size}_seq{maxlen}_mf{max_features}.pkl", "rb") as f:
#     w2v_wordidx_dic = pickle.loads(pickle.load(f))

# with open(f"{UDDIR}/imd/emb_matrix_dic_w2v{embed_size}_seq{maxlen}_mf{max_features}.pkl", "rb") as f:
#     emb_matrix_dic = pickle.loads(pickle.load(f))

In [33]:
gc.collect()

0

## Auxiliary Features

In [ ]:
feat_fname = sorted(os.listdir(UFEDIR))

In [ ]:
train_aux = pd.DataFrame(train_user[UID])
test_aux = pd.DataFrame(test_user[UID])

In [ ]:
for fname in feat_fname:
    if fname.startswith("tfidf_count_emb_"):
        print("current filename: ", fname)
        gender_agg_pred = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_aux = pd.merge(train_aux, gender_agg_pred, how="left", on=UID)
        test_aux = pd.merge(test_aux, gender_agg_pred, how="left", on=UID)
        gender_agg_pred = None

In [ ]:
for fname in feat_fname:
    if fname.startswith("meta_"):
        print("current filename: ", fname)
        gender_agg_pred = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_aux = pd.merge(train_aux, gender_agg_pred, how="left", on=UID)
        test_aux = pd.merge(test_aux, gender_agg_pred, how="left", on=UID)
        gender_agg_pred = None

In [ ]:
# stats cate target encode
for fname in feat_fname:
    if fname.startswith("train_stats_catemlb"):
        print("current filename: ", fname)
        train_aux = pd.merge(train_aux, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_catemlb"):
        print("current filename: ", fname)
        test_aux = pd.merge(test_aux, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
# stats o1
for fname in feat_fname:
    if fname.startswith("train_stats_o1"):
        print("current filename: ", fname)
        train_aux = pd.merge(train_aux, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_o1"):
        print("current filename: ", fname)
        test_aux = pd.merge(test_aux, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
# # stats getidxmax
# for fname in feat_fname:
#     if fname.startswith("train_stats_o2_getidxmax"):
#         print("current filename: ", fname)
#         train_aux = pd.merge(train_aux, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
#     elif fname.startswith("test_stats_o2_getidxmax"):
#         print("current filename: ", fname)
#         test_aux = pd.merge(test_aux, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
for fname in feat_fname:
    if fname.startswith("tfidf_svd_"):
        print("current filename: ", fname)
        cur_tfidf_svd = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_aux = pd.merge(train_aux, cur_tfidf_svd, how="left", on=UID)
        test_aux = pd.merge(test_aux, cur_tfidf_svd, how="left", on=UID)
        cur_tfidf_svd = None

In [ ]:
# # word2vec
# for cur_id in USE_IDS:
#     fname = f"w2v_avg_{cur_id}.pkl"
#     print("current filename: ", fname)
#     cur_df = pd.read_pickle(f"{UFEDIR}/{fname}")
#     train_aux = pd.merge(train_aux, cur_df, how="left", on=UID)
#     test_aux = pd.merge(test_aux, cur_df, how="left", on=UID)

In [ ]:
train_aux.set_index(UID, inplace=True)
test_aux.set_index(UID, inplace=True)

In [ ]:
set(train_aux.columns == test_aux.columns)

In [ ]:
train_aux = train_aux.astype("float32")
test_aux = test_aux.astype("float32")

In [ ]:
def nnaninf(val):
    return np.mean(val[(~np.isnan(val))&(~np.isinf(val))].values)

In [ ]:
def fillmean(df):
    for col in tqdm(df.columns):
        if df[col].count() < len(df):
            df[col] = df[col].replace([np.nan, np.inf], nnaninf(df[col]))
    # check
    for col in tqdm(df.columns):
        if df[col].count() < len(df):
            log(col)
    
    return df

In [ ]:
def fillmean_abs(df):
    for col in tqdm(df.columns):
        u = at_percentile(75)(df[col])
        l = at_percentile(25)(df[col])
        iqr = u - l
        up_bound = u + 1.5*iqr
        low_bound = l - 1.5*iqr
        
        df[col][df[col] < low_bound] = np.mean(df[col].values)
        df[col][df[col] > up_bound] = np.mean(df[col].values)
    # check
    for col in tqdm(df.columns):
        if df[col].count() < len(df):
            log(col)
    
    return df

In [ ]:
train_aux = fillmean(train_aux)
test_aux = fillmean(test_aux)

In [ ]:
train_aux = fillmean_abs(train_aux)
test_aux = fillmean_abs(test_aux)

In [ ]:
def transmms(tr_df, te_df):
    tol_df = pd.concat([tr_df, te_df])
    mms = MinMaxScaler()
    mms_tol_df = mms.fit_transform(tol_df)
    mms_tr_df = mms_tol_df[:len(tr_df)]
    mms_te_df = mms_tol_df[len(tr_df):]
    
    return mms_tr_df, mms_te_df

In [ ]:
train_aux_scal, test_aux_scal = transmms(train_aux, test_aux)

In [ ]:
train_aux_scal.max(), train_aux_scal.min()

In [ ]:
test_aux_scal.max(), test_aux_scal.min()

In [ ]:
# train_aux.to_pickle(f"{UDDIR}/imd/train_aux.pkl")
# test_aux.to_pickle(f"{UDDIR}/imd/test_aux.pkl")
np.save(f"{UDDIR}/imd/train_aux_scal.npy", train_aux_scal)
np.save(f"{UDDIR}/imd/test_aux_scal.npy", test_aux_scal)

In [ ]:
del train_aux, test_aux

In [34]:
train_aux_scal = np.load(f"{UDDIR}/imd/train_aux_scal.npy")
test_aux_scal = np.load(f"{UDDIR}/imd/test_aux_scal.npy")

In [35]:
train_aux_scal.shape

(900000, 1349)

# Model

In [ ]:
np.random.seed(2020)

In [ ]:
# tf.debugging.set_log_device_placement(True)

## ESIM

In [36]:
class SoftAttention(object):
    """
    Layer to compute local inference between two encoded sentences a and b.
    """

    def __call__(self, inputs):
        a = inputs[0]
        b = inputs[1]

        attention = layers.Lambda(self._attention,
                                        output_shape = self._attention_output_shape,
                                        arguments = None)(inputs)

        align_a = layers.Lambda(self._soft_alignment,
                                     output_shape = self._soft_alignment_output_shape,
                                     arguments = None)([attention, a])
        align_b = layers.Lambda(self._soft_alignment,
                                     output_shape = self._soft_alignment_output_shape,
                                     arguments = None)([attention, b])

        return align_a, align_b

    def _attention(self, inputs):
        """
        Compute the attention between elements of two sentences with the dot
        product.
        Args:
            inputs: A list containing two elements, one for the first sentence
                    and one for the second, both encoded by a BiLSTM.
        Returns:
            A tensor containing the dot product (attention weights between the
            elements of the two sentences).
        """
        attn_weights = K.batch_dot(x=inputs[0],
                                   y=K.permute_dimensions(inputs[1],
                                                          pattern=(0, 2, 1)))
        return K.permute_dimensions(attn_weights, (0, 2, 1))

    def _attention_output_shape(self, inputs):
        input_shape = inputs[0]
        embedding_size = input_shape[1]
        return (input_shape[0], embedding_size, embedding_size)

    def _soft_alignment(self, inputs):
        """
        Compute the soft alignment between the elements of two sentences.
        Args:
            inputs: A list of two elements, the first is a tensor of attention
                    weights, the second is the encoded sentence on which to
                    compute the alignments.
        Returns:
            A tensor containing the alignments.
        """
        attention = inputs[0]
        sentence = inputs[1]

        # Subtract the max. from the attention weights to avoid overflows.
        exp = K.exp(attention - K.max(attention, axis=-1, keepdims=True))
        exp_sum = K.sum(exp, axis=-1, keepdims=True)
        softmax = exp / exp_sum

        return K.batch_dot(softmax, sentence)

    def _soft_alignment_output_shape(self, inputs):
        attention_shape = inputs[0]
        sentence_shape = inputs[1]
        return (attention_shape[0], attention_shape[1], sentence_shape[2])


In [37]:
def esim(id1, id2):

    inp_a = layers.Input(shape=(maxlen,))
    inp_b = layers.Input(shape=(maxlen,))
    
    # embedding layer
    embedded_a = layers.Embedding(len(emb_matrix_dic[id1]), embed_size, weights=[emb_matrix_dic[id1]], trainable=False)(inp_a)
    embedded_b = layers.Embedding(len(emb_matrix_dic[id2]), embed_size, weights=[emb_matrix_dic[id2]], trainable=False)(inp_b)
    
    # Encoding Layer
    bilstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.1))
    
    encoded_a = bilstm(embedded_a)
    encoded_b = bilstm(embedded_b)
    
    # local inference layer 
    atten_a, atten_b = SoftAttention()([encoded_a, encoded_b])

    sub_a_atten = layers.Lambda(lambda x: x[0]-x[1])([encoded_a, atten_a])
    sub_b_atten = layers.Lambda(lambda x: x[0]-x[1])([encoded_b, atten_b])

    mul_a_atten = layers.Lambda(lambda x: x[0]*x[1])([encoded_a, atten_a])
    mul_b_atten = layers.Lambda(lambda x: x[0]*x[1])([encoded_b, atten_b])

    m_a = layers.concatenate([encoded_a, atten_a, sub_a_atten, mul_a_atten])
    m_b = layers.concatenate([encoded_b, atten_b, sub_b_atten, mul_b_atten])
    
    # Inference composition layer
    composition_a = layers.Bidirectional(layers.LSTM(
        128,
        return_sequences=True,
        dropout=0.2
    ))(m_a)

    avg_pool_a = layers.GlobalAveragePooling1D()(composition_a)
    max_pool_a = layers.GlobalMaxPooling1D()(composition_a)

    composition_b = layers.Bidirectional(layers.LSTM(
        128,
        return_sequences=True,
        dropout=0.2
    ))(m_b)

    avg_pool_b = layers.GlobalAveragePooling1D()(composition_b)
    max_pool_b = layers.GlobalMaxPooling1D()(composition_b)
    
    
    pooled = layers.concatenate([avg_pool_a, max_pool_a, avg_pool_b, max_pool_b])
    pooled = layers.Dropout(rate=0.1)(pooled)
    
    inp_aux = layers.Input(shape=(train_aux_scal.shape[1], ))
    aux_1 = layers.Dense(32)(inp_aux)
    
    conc = layers.concatenate([pooled, aux_1])
    
    conc = layers.Dense(256, activation="relu")(conc)
    conc = layers.Dense(64, activation="relu")(conc)
    conc = layers.Dense(32, activation="relu")(conc)
    
    conc = layers.Dropout(rate=0.2)(conc)

    out_age = layers.Dense(10, activation="softmax", name="out_age")(conc)
    out_gender = layers.Dense(1, activation="sigmoid", name="out_gender")(conc)

    model = keras.Model(inputs=[inp_a, inp_b, inp_aux], outputs=[out_age, out_gender])
    
    return model
    

In [ ]:
id1 = "creative_id"
id2 = "advertiser_id"

In [ ]:
model = esim(id1, id2)
model.compile(
    loss={"out_age": "categorical_crossentropy", "out_gender": "binary_crossentropy"},
    loss_weights={"out_age": 1, "out_gender": 1},
    optimizer=keras.optimizers.Adam(lr=0.001), 
    metrics=["acc"]
)
model.summary()

In [ ]:
train_set = [train_textprec_dic[id1], train_textprec_dic[id2]]
test_set = [test_textprec_dic[id1], test_textprec_dic[id2]]

In [ ]:
train_y_age = keras.utils.to_categorical(train_user["age"]-1)
train_y_gender = (train_user["gender"]-1).values.reshape(-1, 1)

In [ ]:
res_set = train_test_split(*train_set, train_y_age, train_y_gender, test_size=0.2, random_state=2020)

In [ ]:
train_X_tr = res_set[:-4:2]
train_X_val = res_set[1:-4:2]
train_y_tr_age = res_set[-4]
train_y_val_age = res_set[-3]

train_y_tr_gender = res_set[-2]
train_y_val_gender = res_set[-1]

In [ ]:
elsp = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0)
filepath = f"{UMDIR}/weights_improvement_fold.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only=True, mode="min")

In [ ]:
cb = [elsp,]  # checkpoint]

In [ ]:
model.fit(train_X_tr, [train_y_tr_age, train_y_tr_gender], validation_data=(train_X_val, [train_y_val_age, train_y_val_gender]), epochs=3, callbacks=cb)

In [ ]:
model.save(f"{UMDIR}/5inp_esim_model_emb{embed_size}_seq{maxlen}.h5")

In [ ]:
# model = keras.models.load_model(f"{UMDIR}/5inp_2bilstm_model_128.h5")
model._layers = [
    layer for layer in model._layers if isinstance(layer, keras.layers.Layer)
]
keras.utils.plot_model(model, to_file=f"{UMDIR}/5inp_esim_model_emb{embed_size}_seq{maxlen}.png", show_layer_names=False)

In [ ]:
gc.collect()

In [ ]:
# K-fold
id1 = "creative_id"
id2 = "advertiser_id"

train_set = [train_textprec_dic[id1], train_textprec_dic[id2], train_aux_scal]
test_set = [test_textprec_dic[id1], test_textprec_dic[id2], test_aux_scal]

train_y_age = keras.utils.to_categorical(train_user["age"]-1)
train_y_gender = (train_user["gender"]-1).values.reshape(-1, 1)

DATA_SPLIT_SEED = 2020
nfolds = 5
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=DATA_SPLIT_SEED)

predicted_train_age = np.zeros(train_y_age.shape)
predicted_train_gender = np.zeros(train_y_gender.shape)
predicted_age = np.zeros((len(test_user),train_y_age.shape[1]))
predicted_gender = np.zeros((len(test_user),train_y_gender.shape[1]))

elsp = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0)
cb = [elsp,] 

for idx, (train_idx, valid_idx) in enumerate(kfold.split(train_set[0])):
    print("Current fold:", idx+1)
    
    K.clear_session()
    gc.collect()
    
    X_train_set = list()
    X_val_set = list()
    
    for cur_train in train_set:
        X_train_set.append(cur_train[train_idx])
        X_val_set.append(cur_train[valid_idx])
        
    y_train_age = train_y_age[train_idx]
    y_train_gender = train_y_gender[train_idx]
    
    y_val_age = train_y_age[valid_idx]
    y_val_gender = train_y_gender[valid_idx]
    

    model = esim(id1, id2)
    model.compile(
        loss={"out_age": "categorical_crossentropy", "out_gender": "binary_crossentropy"},
        loss_weights={"out_age": 1, "out_gender": 1},
        optimizer=keras.optimizers.Adam(lr=0.001), 
        metrics=["acc"]
    )
    model.summary()
    
    model.fit(X_train_set, [y_train_age, y_train_gender], validation_data=(X_val_set, [y_val_age, y_val_gender]), batch_size=1024, epochs=50, callbacks=cb)
    
    predicted_train_age[valid_idx], predicted_train_gender[valid_idx] = model.predict(X_val_set, batch_size=1024, verbose=1)
    pred_test_y_age, pred_test_y_gender = model.predict(test_set, batch_size=1024, verbose=1)
    predicted_age += pred_test_y_age / nfolds
    predicted_gender += pred_test_y_gender / nfolds

Current fold: 1
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 128, 128)     436834944   input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 128, 128)     7407488     input_2[0][0]                    
______________________________________________________________________________

Epoch 1/50
704/704 [==============================] - 4248s 6s/step - loss: 1.7229 - out_age_loss: 1.5111 - out_gender_loss: 0.2118 - out_age_acc: 0.3860 - out_gender_acc: 0.9248 - val_loss: 1.5243 - val_out_age_loss: 1.3546 - val_out_gender_loss: 0.1697 - val_out_age_acc: 0.4407 - val_out_gender_acc: 0.9387
Epoch 2/50
704/704 [==============================] - 4169s 6s/step - loss: 1.5510 - out_age_loss: 1.3704 - out_gender_loss: 0.1806 - out_age_acc: 0.4370 - out_gender_acc: 0.9387 - val_loss: 1.4852 - val_out_age_loss: 1.3250 - val_out_gender_loss: 0.1602 - val_out_age_acc: 0.4505 - val_out_gender_acc: 0.9440
Epoch 3/50
704/704 [==============================] - 4164s 6s/step - loss: 1.5170 - out_age_loss: 1.3431 - out_gender_loss: 0.1740 - out_age_acc: 0.4470 - out_gender_acc: 0.9412 - val_loss: 1.4693 - val_out_age_loss: 1.3093 - val_out_gender_loss: 0.1600 - val_out_age_acc: 0.4556 - val_out_gender_acc: 0.9446
Epoch 4/50
704/704 [==============================] - 4119s 6s/step - 

In [ ]:
accuracy_score(np.argmax(train_y_age, axis=1), np.argmax(predicted_train_age, axis=1))

In [ ]:
accuracy_score(train_y_gender, (predicted_train_gender >= 0.5).astype(int))

## LSTM-Class

In [ ]:
class Attention(layers.Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = keras.initializers.get('glorot_uniform')

        self.W_regularizer = keras.regularizers.get(W_regularizer)
        self.b_regularizer = keras.regularizers.get(b_regularizer)

        self.W_constraint = keras.constraints.get(W_constraint)
        self.b_constraint = keras.constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
    

In [ ]:
class ScaledDotProductAttention(keras.layers.Layer):
    r"""The attention layer that takes three inputs representing queries, keys and values.
    \text{Attention}(Q, K, V) = \text{softmax}(\frac{Q K^T}{\sqrt{d_k}}) V
    See: https://arxiv.org/pdf/1706.03762.pdf
    """

    def __init__(self,
                 return_attention=False,
                 history_only=False,
                 **kwargs):
        """Initialize the layer.
        :param return_attention: Whether to return attention weights.
        :param history_only: Whether to only use history data.
        :param kwargs: Arguments for parent class.
        """
        super(ScaledDotProductAttention, self).__init__(**kwargs)
        self.supports_masking = True
        self.return_attention = return_attention
        self.history_only = history_only
        self.intensity = self.attention = None

    def get_config(self):
        config = {
            'return_attention': self.return_attention,
            'history_only': self.history_only,
        }
        base_config = super(ScaledDotProductAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            query_shape, key_shape, value_shape = input_shape
        else:
            query_shape = key_shape = value_shape = input_shape
        output_shape = query_shape[:-1] + value_shape[-1:]
        if self.return_attention:
            attention_shape = query_shape[:2] + (key_shape[1],)
            return [output_shape, attention_shape]
        return output_shape

    def compute_mask(self, inputs, mask=None):
        if isinstance(mask, list):
            mask = mask[0]
        if self.return_attention:
            return [mask, None]
        return mask

    def call(self, inputs, mask=None, **kwargs):
        if isinstance(inputs, list):
            query, key, value = inputs
        else:
            query = key = value = inputs
        if isinstance(mask, list):
            mask = mask[1]
        feature_dim = K.shape(query)[-1]
        e = K.batch_dot(query, key, axes=2) / K.sqrt(K.cast(feature_dim, dtype=K.floatx()))
        if self.history_only:
            query_len, key_len = K.shape(query)[1], K.shape(key)[1]
            indices = K.expand_dims(K.arange(0, key_len), axis=0)
            upper = K.expand_dims(K.arange(0, query_len), axis=-1)
            e -= 10000.0 * K.expand_dims(K.cast(indices > upper, K.floatx()), axis=0)
        if mask is not None:
            e -= 10000.0 * (1.0 - K.cast(K.expand_dims(mask, axis=-2), K.floatx()))
        self.intensity = e
        e = K.exp(e - K.max(e, axis=-1, keepdims=True))
        self.attention = e / K.sum(e, axis=-1, keepdims=True)
        v = K.batch_dot(self.attention, value)
        if self.return_attention:
            return [v, self.attention]
        return v

In [ ]:
class SeqSelfAttention(keras.layers.Layer):

    ATTENTION_TYPE_ADD = 'additive'
    ATTENTION_TYPE_MUL = 'multiplicative'

    def __init__(self,
                 units=32,
                 attention_width=None,
                 attention_type=ATTENTION_TYPE_ADD,
                 return_attention=False,
                 history_only=False,
                 kernel_initializer='glorot_normal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 use_additive_bias=True,
                 use_attention_bias=True,
                 attention_activation=None,
                 attention_regularizer_weight=0.0,
                 **kwargs):
        """Layer initialization.
        For additive attention, see: https://arxiv.org/pdf/1806.01264.pdf
        :param units: The dimension of the vectors that used to calculate the attention weights.
        :param attention_width: The width of local attention.
        :param attention_type: 'additive' or 'multiplicative'.
        :param return_attention: Whether to return the attention weights for visualization.
        :param history_only: Only use historical pieces of data.
        :param kernel_initializer: The initializer for weight matrices.
        :param bias_initializer: The initializer for biases.
        :param kernel_regularizer: The regularization for weight matrices.
        :param bias_regularizer: The regularization for biases.
        :param kernel_constraint: The constraint for weight matrices.
        :param bias_constraint: The constraint for biases.
        :param use_additive_bias: Whether to use bias while calculating the relevance of inputs features
                                  in additive mode.
        :param use_attention_bias: Whether to use bias while calculating the weights of attention.
        :param attention_activation: The activation used for calculating the weights of attention.
        :param attention_regularizer_weight: The weights of attention regularizer.
        :param kwargs: Parameters for parent class.
        """
        super(SeqSelfAttention, self).__init__(**kwargs)
        self.supports_masking = True
        self.units = units
        self.attention_width = attention_width
        self.attention_type = attention_type
        self.return_attention = return_attention
        self.history_only = history_only
        if history_only and attention_width is None:
            self.attention_width = int(1e9)

        self.use_additive_bias = use_additive_bias
        self.use_attention_bias = use_attention_bias
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)
        self.kernel_constraint = keras.constraints.get(kernel_constraint)
        self.bias_constraint = keras.constraints.get(bias_constraint)
        self.attention_activation = keras.activations.get(attention_activation)
        self.attention_regularizer_weight = attention_regularizer_weight
        self._backend = keras.backend.backend()

        if attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self.Wx, self.Wt, self.bh = None, None, None
            self.Wa, self.ba = None, None
        elif attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self.Wa, self.ba = None, None
        else:
            raise NotImplementedError('No implementation for attention type : ' + attention_type)

    def get_config(self):
        config = {
            'units': self.units,
            'attention_width': self.attention_width,
            'attention_type': self.attention_type,
            'return_attention': self.return_attention,
            'history_only': self.history_only,
            'use_additive_bias': self.use_additive_bias,
            'use_attention_bias': self.use_attention_bias,
            'kernel_initializer': keras.initializers.serialize(self.kernel_initializer),
            'bias_initializer': keras.initializers.serialize(self.bias_initializer),
            'kernel_regularizer': keras.regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': keras.regularizers.serialize(self.bias_regularizer),
            'kernel_constraint': keras.constraints.serialize(self.kernel_constraint),
            'bias_constraint': keras.constraints.serialize(self.bias_constraint),
            'attention_activation': keras.activations.serialize(self.attention_activation),
            'attention_regularizer_weight': self.attention_regularizer_weight,
        }
        base_config = super(SeqSelfAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self._build_additive_attention(input_shape)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self._build_multiplicative_attention(input_shape)
        super(SeqSelfAttention, self).build(input_shape)

    def _build_additive_attention(self, input_shape):
        feature_dim = int(input_shape[2])

        self.Wt = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wt'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        self.Wx = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wx'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_additive_bias:
            self.bh = self.add_weight(shape=(self.units,),
                                      name='{}_Add_bh'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

        self.Wa = self.add_weight(shape=(self.units, 1),
                                  name='{}_Add_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Add_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

    def _build_multiplicative_attention(self, input_shape):
        feature_dim = int(input_shape[2])

        self.Wa = self.add_weight(shape=(feature_dim, feature_dim),
                                  name='{}_Mul_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Mul_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

    def call(self, inputs, mask=None, **kwargs):
        input_len = K.shape(inputs)[1]

        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            e = self._call_additive_emission(inputs)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            e = self._call_multiplicative_emission(inputs)

        if self.attention_activation is not None:
            e = self.attention_activation(e)
        if self.attention_width is not None:
            if self.history_only:
                lower = K.arange(0, input_len) - (self.attention_width - 1)
            else:
                lower = K.arange(0, input_len) - self.attention_width // 2
            lower = K.expand_dims(lower, axis=-1)
            upper = lower + self.attention_width
            indices = K.expand_dims(K.arange(0, input_len), axis=0)
            e -= 10000.0 * (K.cast(indices < lower, K.floatx()) * K.cast(upper <= indices, K.floatx()))
        if mask is not None:
            mask = K.expand_dims(K.cast(mask, K.floatx()), axis=-1)
            e -= 10000.0 * ((1.0 - mask) * (1.0 - K.permute_dimensions(mask, (0, 2, 1))))

        # a_{t} = \text{softmax}(e_t)
        e = K.exp(e - K.max(e, axis=-1, keepdims=True))
        a = e / K.sum(e, axis=-1, keepdims=True)

        # l_t = \sum_{t'} a_{t, t'} x_{t'}
        v = K.batch_dot(a, inputs)
        if self.attention_regularizer_weight > 0.0:
            self.add_loss(self._attention_regularizer(a))

        if self.return_attention:
            return [v, a]
        return v

    def _call_additive_emission(self, inputs):
        input_shape = K.shape(inputs)
        batch_size, input_len = input_shape[0], input_shape[1]

        # h_{t, t'} = \tanh(x_t^T W_t + x_{t'}^T W_x + b_h)
        q = K.expand_dims(K.dot(inputs, self.Wt), 2)
        k = K.expand_dims(K.dot(inputs, self.Wx), 1)
        if self.use_additive_bias:
            h = K.tanh(q + k + self.bh)
        else:
            h = K.tanh(q + k)

        # e_{t, t'} = W_a h_{t, t'} + b_a
        if self.use_attention_bias:
            e = K.reshape(K.dot(h, self.Wa) + self.ba, (batch_size, input_len, input_len))
        else:
            e = K.reshape(K.dot(h, self.Wa), (batch_size, input_len, input_len))
        return e

    def _call_multiplicative_emission(self, inputs):
        # e_{t, t'} = x_t^T W_a x_{t'} + b_a
        e = K.batch_dot(K.dot(inputs, self.Wa), K.permute_dimensions(inputs, (0, 2, 1)))
        if self.use_attention_bias:
            e += self.ba[0]
        return e

    def compute_output_shape(self, input_shape):
        output_shape = input_shape
        if self.return_attention:
            attention_shape = (input_shape[0], output_shape[1], input_shape[1])
            return [output_shape, attention_shape]
        return output_shape

    def compute_mask(self, inputs, mask=None):
        if self.return_attention:
            return [mask, None]
        return mask

    def _attention_regularizer(self, attention):
        batch_size = K.cast(K.shape(attention)[0], K.floatx())
        input_len = K.shape(attention)[-1]
        indices = K.expand_dims(K.arange(0, input_len), axis=0)
        diagonal = K.expand_dims(K.arange(0, input_len), axis=-1)
        eye = K.cast(K.equal(indices, diagonal), K.floatx())
        return self.attention_regularizer_weight * K.sum(K.square(K.batch_dot(
            attention,
            K.permute_dimensions(attention, (0, 2, 1))) - eye)) / batch_size

    @staticmethod
    def get_custom_objects():
        return {'SeqSelfAttention': SeqSelfAttention}

In [ ]:
class SeqWeightedAttention(keras.layers.Layer):
    r"""Y = \text{softmax}(XW + b) X
    See: https://arxiv.org/pdf/1708.00524.pdf
    """

    def __init__(self, use_bias=True, return_attention=False, **kwargs):
        super(SeqWeightedAttention, self).__init__(**kwargs)
        self.supports_masking = True
        self.use_bias = use_bias
        self.return_attention = return_attention
        self.W, self.b = None, None

    def get_config(self):
        config = {
            'use_bias': self.use_bias,
            'return_attention': self.return_attention,
        }
        base_config = super(SeqWeightedAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        self.W = self.add_weight(shape=(int(input_shape[2]), 1),
                                 name='{}_W'.format(self.name),
                                 initializer=keras.initializers.get('uniform'))
        if self.use_bias:
            self.b = self.add_weight(shape=(1,),
                                     name='{}_b'.format(self.name),
                                     initializer=keras.initializers.get('zeros'))
        super(SeqWeightedAttention, self).build(input_shape)

    def call(self, x, mask=None):
        logits = K.dot(x, self.W)
        if self.use_bias:
            logits += self.b
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            logits -= 10000.0 * (1.0 - mask)
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return input_shape[0], output_len

    def compute_mask(self, _, input_mask=None):
        if self.return_attention:
            return [None, None]
        return None

    @staticmethod
    def get_custom_objects():
        return {'SeqWeightedAttention': SeqWeightedAttention}

In [ ]:
USE_IDS

In [ ]:
id1 = "creative_id"
id2 = "ad_id"
id3 = "advertiser_id"
# id4 = "product_id"
# id5 = "industry"
# id6 = "product_category"

In [ ]:
inp_1 = layers.Input(shape=(maxlen,))
inp_2 = layers.Input(shape=(maxlen,))
inp_3 = layers.Input(shape=(maxlen,))
# inp_4 = layers.Input(shape=(maxlen,))
# inp_5 = layers.Input(shape=(maxlen,))
# inp_6 = layers.Input(shape=(maxlen,))

In [ ]:
embed_1 = layers.Embedding(input_dim=len(emb_matrix_dic[id1]), output_dim=embed_size, weights=[emb_matrix_dic[id1]], trainable=False)(inp_1)
embed_2 = layers.Embedding(input_dim=len(emb_matrix_dic[id2]), output_dim=embed_size, weights=[emb_matrix_dic[id2]], trainable=False)(inp_2)
embed_3 = layers.Embedding(input_dim=len(emb_matrix_dic[id3]), output_dim=embed_size, weights=[emb_matrix_dic[id3]], trainable=False)(inp_3)
# embed_4 = layers.Embedding(input_dim=len(emb_matrix_dic[id4]), output_dim=embed_size, weights=[emb_matrix_dic[id4]], trainable=False)(inp_4)
# embed_5 = layers.Embedding(input_dim=len(emb_matrix_dic[id5]), output_dim=embed_size, weights=[emb_matrix_dic[id5]], trainable=False)(inp_5)
# embed_6 = layers.Embedding(input_dim=len(emb_matrix_dic[id6]), output_dim=embed_size, weights=[emb_matrix_dic[id6]], trainable=False)(inp_6)

In [ ]:
bilstm_1 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_1)
# gru_1 = layers.Bidirectional(layers.GRU(64, return_sequences=True))(bilstm_1)
bilstm_2 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_2)
# gru_2 = layers.Bidirectional(layers.GRU(64, return_sequences=True))(bilstm_2)
bilstm_3 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_3)
# gru_3 = layers.Bidirectional(layers.GRU(64, return_sequences=True))(bilstm_3)
# bilstm_4 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_4)
# gru_4 = layers.Bidirectional(layers.GRU(64, return_sequences=True))(bilstm_4)
# bilstm_5 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_5)
# gru_5 = layers.Bidirectional(layers.GRU(64, return_sequences=True))(bilstm_5)
# bilstm_6 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_6)
# bilstm_6 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(bilstm_6)

In [ ]:
# bilstm_1 = layers.Bidirectional(layers.LSTM(128))(embed_1)
# bilstm_2 = layers.Bidirectional(layers.LSTM(128))(embed_2)
# bilstm_3 = layers.Bidirectional(layers.LSTM(128))(embed_3)

In [ ]:
# atten_1 = Attention(maxlen)(bilstm_1)
# atten_11 = Attention(maxlen)(gru_1)
# atten_2 = Attention(maxlen)(bilstm_2)
# atten_22 = Attention(maxlen)(gru_2)
# atten_3 = Attention(maxlen)(bilstm_3)
# atten_33 = Attention(maxlen)(gru_3)
# atten_4 = Attention(maxlen)(bilstm_4)
# atten_44 = Attention(maxlen)(gru_4)
# atten_5 = Attention(maxlen)(bilstm_5)
# atten_55 = Attention(maxlen)(gru_5)
# atten_6 = Attention(maxlen)(bilstm_6)

In [ ]:
# maxpool_1 = layers.GlobalMaxPooling1D()(gru_1)
# maxpool_2 = layers.GlobalMaxPooling1D()(gru_2)
# maxpool_3 = layers.GlobalMaxPooling1D()(gru_3)
# maxpool_4 = layers.GlobalMaxPooling1D()(gru_4)
# maxpool_5 = layers.GlobalMaxPooling1D()(gru_5)
# maxpool_6 = layers.GlobalMaxPooling1D()(bilstm_6)

In [ ]:
# inp_aux = layers.Input(shape=(train_aux_scal.shape[1], ))
# aux_1 = layers.Dense(128)(inp_aux)
# aux_1 = layers.BatchNormalization()(aux_1)

In [ ]:
# conc_list = [maxpool_1, maxpool_2, maxpool_3, ]  #  maxpool_4, maxpool_5, aux_1]
# conc_list = [bilstm_1, bilstm_2, bilstm_3]

In [ ]:
# conc_list = [atten_1, atten_11, maxpool_1, atten_2, atten_22, maxpool_2, atten_3, atten_33, maxpool_3, atten_4, atten_44, maxpool_4, atten_5, atten_55, maxpool_5, aux_1]

In [ ]:
# conc = layers.concatenate(conc_list)

In [ ]:
bilstm_conc = layers.concatenate([bilstm_1, bilstm_2, bilstm_3])

In [ ]:
bilstm_conc.shape

In [ ]:
atten_1 = ScaledDotProductAttention()(bilstm_conc)

In [ ]:
atten_1

In [ ]:
maxpool_1 = layers.GlobalMaxPooling1D()(atten_1)

In [ ]:
maxpool_1

In [ ]:
conc = maxpool_1

In [ ]:
# conc = layers.Dense(256)(atten_1)
# conc = layers.BatchNormalization()(conc)
# conc = layers.PReLU()(conc)
# conc = layers.Dropout(0.2)(conc)

# conc = layers.Dense(128)(conc)
# conc = layers.BatchNormalization()(conc)

# out_age = layers.Dense(10, activation="softmax", name="out_age")(conc)
# out_gender = layers.Dense(1, activation="sigmoid", name="out_gender")(conc)

conc = layers.Dense(256, activation="relu")(conc)
conc = layers.Dense(64, activation="relu")(conc)
conc = layers.Dense(32, activation="relu")(conc)

conc = layers.Dropout(rate=0.2)(conc)

out_age = layers.Dense(10, activation="softmax", name="out_age")(conc)
out_gender = layers.Dense(1, activation="sigmoid", name="out_gender")(conc)

model = keras.Model(inputs=[inp_1, inp_2, inp_3, ], outputs=[out_age, out_gender])  #  inp_4, inp_5, inp_aux

In [ ]:
# current lr=0.0005
model.compile(
    loss={"out_age": "categorical_crossentropy", "out_gender": "binary_crossentropy"},
    loss_weights={"out_age": 1, "out_gender": 1},
    optimizer=tfa.optimizers.RectifiedAdam(lr=0.001), 
    metrics=["acc"]
)

In [ ]:
model.summary()

In [ ]:
# help(tfa.optimizers.RectifiedAdam)

In [ ]:
train_set = [train_textprec_dic[id1], train_textprec_dic[id2], train_textprec_dic[id3],]   # train_textprec_dic[id1], train_textprec_dic[id2], train_textprec_dic[id3], train_textprec_dic[id4], train_textprec_dic[id5], train_aux_scal]
test_set = [test_textprec_dic[id1], test_textprec_dic[id2], test_textprec_dic[id3],]   # test_textprec_dic[id1], test_textprec_dic[id2], test_textprec_dic[id3], test_textprec_dic[id4], test_textprec_dic[id5], test_aux_scal]

In [ ]:
train_y_age = keras.utils.to_categorical(train_user["age"]-1)
train_y_gender = (train_user["gender"]-1).values.reshape(-1, 1)

In [ ]:
res_set = train_test_split(*train_set, train_y_age, train_y_gender, test_size=0.2, random_state=2020)

In [ ]:
train_X_tr = res_set[:-4:2]
train_X_val = res_set[1:-4:2]
train_y_tr_age = res_set[-4]
train_y_val_age = res_set[-3]

train_y_tr_gender = res_set[-2]
train_y_val_gender = res_set[-1]

In [ ]:
elsp = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0)
filepath = f"{UMDIR}/weights_improvement_fold.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only=True, mode="min")

In [ ]:
cb = [elsp,]  # checkpoint]

In [ ]:
model.fit(train_X_tr, [train_y_tr_age, train_y_tr_gender], validation_data=(train_X_val, [train_y_val_age, train_y_val_gender]), batch_size=1024, epochs=10, callbacks=cb)

In [ ]:
Epoch 1/20
22500/22500 [==============================] - 2096s 93ms/step - loss: 1.6319 - out_age_loss: 1.4304 - out_gender_loss: 0.2015 - out_age_acc: 0.4164 - out_gender_acc: 0.9272 - val_loss: 1.4971 - val_out_age_loss: 1.3261 - val_out_gender_loss: 0.1710 - val_out_age_acc: 0.4511 - val_out_gender_acc: 0.9396
Epoch 2/20
22500/22500 [==============================] - 2092s 93ms/step - loss: 1.5176 - out_age_loss: 1.3389 - out_gender_loss: 0.1787 - out_age_acc: 0.4474 - out_gender_acc: 0.9367 - val_loss: 1.4781 - val_out_age_loss: 1.3116 - val_out_gender_loss: 0.1666 - val_out_age_acc: 0.4571 - val_out_gender_acc: 0.9408
Epoch 3/20
22500/22500 [==============================] - 2094s 93ms/step - loss: 1.4770 - out_age_loss: 1.3061 - out_gender_loss: 0.1709 - out_age_acc: 0.4607 - out_gender_acc: 0.9398 - val_loss: 1.4576 - val_out_age_loss: 1.2924 - val_out_gender_loss: 0.1652 - val_out_age_acc: 0.4647 - val_out_gender_acc: 0.9412
Epoch 4/20
22500/22500 [==============================] - 2099s 93ms/step - loss: 1.4410 - out_age_loss: 1.2765 - out_gender_loss: 0.1645 - out_age_acc: 0.4714 - out_gender_acc: 0.9421 - val_loss: 1.4564 - val_out_age_loss: 1.2911 - val_out_gender_loss: 0.1652 - val_out_age_acc: 0.4643 - val_out_gender_acc: 0.9415

In [ ]:
model.save(f"{UMDIR}/3inp_2bilstm_model_emb{embed_size}_seq{maxlen}.h5")

In [ ]:
# model = keras.model.load_model(filepath)
# pred_res = model.predict(test_set, verbose=1)

In [ ]:
model.evaluate(train_X_val, [train_y_val_age, train_y_val_gender])

In [ ]:
Epoch 1/3
22500/22500 [==============================] - 127s 6ms/step - loss: 1.6434 - out_age_loss: 1.4391 - out_gender_loss: 0.2043 - out_age_acc: 0.4136 - out_gender_acc: 0.9259 - val_loss: 1.5569 - val_out_age_loss: 1.3704 - val_out_gender_loss: 0.1865 - val_out_age_acc: 0.4366 - val_out_gender_acc: 0.9329
Epoch 2/3
22500/22500 [==============================] - 127s 6ms/step - loss: 1.5836 - out_age_loss: 1.3904 - out_gender_loss: 0.1932 - out_age_acc: 0.4306 - out_gender_acc: 0.9311 - val_loss: 1.5369 - val_out_age_loss: 1.3560 - val_out_gender_loss: 0.1809 - val_out_age_acc: 0.4392 - val_out_gender_acc: 0.9349
Epoch 3/3
22500/22500 [==============================] - 128s 6ms/step - loss: 1.5697 - out_age_loss: 1.3798 - out_gender_loss: 0.1900 - out_age_acc: 0.4348 - out_gender_acc: 0.9324 - val_loss: 1.5385 - val_out_age_loss: 1.3586 - val_out_gender_loss: 0.1798 - val_out_age_acc: 0.4391 - val_out_gender_acc: 0.9349
<tensorflow.python.keras.callbacks.History at 0x7f1870d90cf8>

In [ ]:
22500/22500 [==============================] - 126s 6ms/step - loss: 1.5604 - out_age_loss: 1.3715 - out_gender_loss: 0.1889 - out_age_acc: 0.4366 - out_gender_acc: 0.9328 - val_loss: 1.5267 - val_out_age_loss: 1.3472 - val_out_gender_loss: 0.1795 - val_out_age_acc: 0.4420 - val_out_gender_acc: 0.9356

In [ ]:
model = keras.models.load_model(f"{UMDIR}/3inp_2bilstm_model_emb{embed_size}_seq{maxlen}.h5")

In [ ]:
pred_res = model.predict(test_set, verbose=1)

In [ ]:
## Plot

In [ ]:
# model = keras.models.load_model(f"{UMDIR}/3inp_2bilstm_model_emb{embed_size}_seq{maxlen}.h5")
# model = keras.models.load_model(f"{UMDIR}/5inp_2bilstm_model_128.h5")
model._layers = [
    layer for layer in model._layers if isinstance(layer, keras.layers.Layer)
]
keras.utils.plot_model(model, to_file=f"{UMDIR}/3inp_2bilstm_model_emb{embed_size}_seq{maxlen}.png", show_layer_names=False)

In [ ]:
# K-fold
id1 = "creative_id"
id2 = "ad_id"
id3 = "advertiser_id"


train_set = [train_textprec_dic[id1], train_textprec_dic[id2], train_textprec_dic[id3],]
test_set = [test_textprec_dic[id1], test_textprec_dic[id2], test_textprec_dic[id3],]  

train_y_age = keras.utils.to_categorical(train_user["age"]-1)
train_y_gender = (train_user["gender"]-1).values.reshape(-1, 1)

DATA_SPLIT_SEED = 2020
nfolds = 5
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=DATA_SPLIT_SEED)

predicted_train_age = np.zeros(train_y_age.shape)
predicted_train_gender = np.zeros(train_y_gender.shape)
predicted_age = np.zeros((len(test_user),train_y_age.shape[1]))
predicted_gender = np.zeros((len(test_user),train_y_gender.shape[1]))

elsp = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0)
cb = [elsp,] 

for idx, (train_idx, valid_idx) in enumerate(kfold.split(train_set[0])):
    print("Current fold:", idx+1)
    K.clear_session()
    gc.collect()
    
    X_train_set = list()
    X_val_set = list()
    
    for cur_train in train_set:
        X_train_set.append(cur_train[train_idx])
        X_val_set.append(cur_train[valid_idx])
        
    y_train_age = train_y_age[train_idx]
    y_train_gender = train_y_gender[train_idx]
    
    y_val_age = train_y_age[valid_idx]
    y_val_gender = train_y_gender[valid_idx]
    
    inp_1 = layers.Input(shape=(maxlen,))
    inp_2 = layers.Input(shape=(maxlen,))
    inp_3 = layers.Input(shape=(maxlen,))
    
    embed_1 = layers.Embedding(input_dim=len(emb_matrix_dic[id1]), output_dim=embed_size, weights=[emb_matrix_dic[id1]], trainable=False)(inp_1)
    embed_2 = layers.Embedding(input_dim=len(emb_matrix_dic[id2]), output_dim=embed_size, weights=[emb_matrix_dic[id2]], trainable=False)(inp_2)
    embed_3 = layers.Embedding(input_dim=len(emb_matrix_dic[id3]), output_dim=embed_size, weights=[emb_matrix_dic[id3]], trainable=False)(inp_3)

    bilstm_1 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_1)
    bilstm_2 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_2)
    bilstm_3 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(embed_3)
    
    bilstm_conc = layers.concatenate([bilstm_1, bilstm_2, bilstm_3])
    atten_1 = Attention()(bilstm_conc)
#     atten_1 = Attention(maxlen)(bilstm_1)
#     atten_2 = Attention(maxlen)(bilstm_2)
#     atten_3 = Attention(maxlen)(bilstm_3)
    
#     conc_list = [atten_1, atten_2, atten_3]
    
#     conc = layers.concatenate(conc_list)
    conc = layers.Dense(256, activation="relu")(atten_1)
    conc = layers.Dense(64, activation="relu")(conc)
    conc = layers.Dense(32, activation="relu")(conc)
    
    conc = layers.Dropout(rate=0.2)(conc)
    
    out_age = layers.Dense(10, activation="softmax", name="out_age")(conc)
    out_gender = layers.Dense(1, activation="sigmoid", name="out_gender")(conc)

    model = keras.Model(inputs=[inp_1, inp_2, inp_3, ], outputs=[out_age, out_gender])
    
    model.compile(
        loss={"out_age": "categorical_crossentropy", "out_gender": "binary_crossentropy"},
        loss_weights={"out_age": 1, "out_gender": 1},
        optimizer=keras.optimizers.Adam(lr=0.001), 
        metrics=["acc"]
    )
    model.summary()
    
    model.fit(X_train_set, [y_train_age, y_train_gender], validation_data=(X_val_set, [y_val_age, y_val_gender]), batch_size=1024, epochs=50, callbacks=cb)
    
    predicted_train_age[valid_idx], predicted_train_gender[valid_idx] = model.predict(X_val_set, batch_size=1024, verbose=1)
    pred_test_y_age, pred_test_y_gender = model.predict(test_set, batch_size=1024, verbose=1)
    predicted_age += pred_test_y_age / nfolds
    predicted_gender += pred_test_y_gender / nfolds

## BERT Transformers 

In [ ]:
from transformers import BertTokenizer
from transformers.modeling_tf_bert import (BertConfig, TFBertModel,
                                           TFBertEmbeddings, TFBertMainLayer,
                                           TFBertEncoder, TFBertAttention, 
                                           TFBertSelfAttention, TFBertLayer, TFBertPooler,
                                           TFBertIntermediate, TFBertOutput,
                                           TFBertForSequenceClassification, TFBertSelfOutput
                                          )

In [ ]:
train_cts = pd.read_pickle(f"{UDDIR}/imd/train_click_times_seq.pkl")
test_cts = pd.read_pickle(f"{UDDIR}/imd/test_click_times_seq.pkl")

In [ ]:
train_mask = keras.preprocessing.sequence.pad_sequences(train_cts["click_times"].values, maxlen=maxlen, padding="post")
test_mask = keras.preprocessing.sequence.pad_sequences(test_cts["click_times"].values, maxlen=maxlen, padding="post")

In [ ]:
# position ids
train_pos_ids = dict()
for col in train_textprec_dic:
    lines, cols = train_textprec_dic[col].shape
    train_pos_ids[col] = np.array([list(range(cols))]*lines)

test_pos_ids = dict()
for col in test_textprec_dic:
    lines, cols = test_textprec_dic[col].shape
    test_pos_ids[col] = np.array([list(range(cols))]*lines)

In [ ]:
# token type ids
train_token_ids = dict()
for col in train_textprec_dic:
    lines, cols = train_textprec_dic[col].shape
    train_token_ids[col] = np.array([[0]*cols]*lines)

test_token_ids = dict()
for col in test_textprec_dic:
    lines, cols = test_textprec_dic[col].shape
    test_token_ids[col] = np.array([[0]*cols]*lines)

In [ ]:
id1 = "creative_id"
id2 = "ad_id"
id3 = "advertiser_id"
id4 = "product_id"
id5 = "industry"
# id6 = "product_category"
TFBS = [id1, id2, id4]

In [ ]:
## My own transformer

In [ ]:
def shape_list(x):
    """Deal with dynamic shape in tensorflow cleanly."""
    static = x.shape.as_list()
    dynamic = tf.shape(x)
    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

In [ ]:
def get_initializer(initializer_range=0.02):
    """Creates a `tf.initializers.truncated_normal` with the given range.
    Args:
        initializer_range: float, initializer range for stddev.
    Returns:
        TruncatedNormal initializer with stddev = `initializer_range`.
    """
    return keras.initializers.TruncatedNormal(stddev=initializer_range)

In [ ]:
def cast_bool_to_primitive(bool_variable, default_tensor_to_true=False):
    """Function arguments can be inserted as boolean tensor
        and bool variables to cope with keras serialization
        we need to cast `output_attentions` to correct bool
        if it is a tensor

    Args:
        default_tensor_to_true: bool, if tensor should default to True
        in case tensor has no numpy attribute
    """
    # if bool variable is tensor and has numpy value
    if tf.is_tensor(bool_variable):
        if hasattr(bool_variable, "numpy"):
            return bool(bool_variable.numpy())
        elif default_tensor_to_true:
            return True

    # else variable is bool
    return bool_variable

In [ ]:
def gelu(x):
    """ Gaussian Error Linear Unit.
    Original Implementation of the gelu activation function in Google Bert repo when initially created.
        For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
        0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
        Also see https://arxiv.org/abs/1606.08415
    """
    cdf = 0.5 * (1.0 + tf.math.erf(x / tf.math.sqrt(2.0)))
    return x * cdf


def gelu_new(x):
    """Gaussian Error Linear Unit.
    This is a smoother version of the RELU.
    Original paper: https://arxiv.org/abs/1606.08415
    Args:
        x: float Tensor to perform activation.
    Returns:
        `x` with the GELU activation applied.
    """
    cdf = 0.5 * (1.0 + tf.tanh((np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf


def swish(x):
    return x * tf.sigmoid(x)

ACT2FN = {
    "gelu": tf.keras.layers.Activation(gelu),
    "relu": tf.keras.activations.relu,
    "swish": tf.keras.layers.Activation(swish),
    "gelu_new": tf.keras.layers.Activation(gelu_new),
}

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # 将 sin 应用于数组中的偶数索引（indices）；2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # 将 cos 应用于数组中的奇数索引；2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
### Input Embedding

In [ ]:
def MyTFBertEmbedding(emb_matrix, config):
    # input ids embedding
    inp_ids = layers.Input(shape=(maxlen,))
    inputs_embeds = layers.Embedding(input_dim=len(emb_matrix), output_dim=embed_size, weights=[emb_matrix], trainable=False)(inp_ids)
    
    # input position Embedding
    inp_pos = layers.Input(shape=(maxlen,))
    position_embeds = layers.Embedding(input_dim=config.max_position_embeddings, output_dim=embed_size, weights=positional_encoding(config.max_position_embeddings, embed_size), trainable=False)(inp_pos)
    
    # input token type Embedding
    inp_token_type = layers.Input(shape=(maxlen,))
    token_type_embeds = layers.Embedding(input_dim=config.type_vocab_size, output_dim=embed_size, embeddings_initializer=get_initializer(config.initializer_range))(inp_token_type)
    
    embeds = inputs_embeds + position_embeds + token_type_embeds
    embeds = layers.LayerNormalization(epsilon=config.layer_norm_eps)(embeds)
    embeds = layers.Dropout(config.hidden_dropout_prob)(embeds)
    
    ret = {"inputs": [inp_ids, inp_pos, inp_token_type],
           "embeddings": embeds
          }
    
    return ret

In [ ]:
## Encoder include encoder layer

In [ ]:
class TFBertSelfAttention(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (config.hidden_size, config.num_attention_heads)
            )

        self.num_attention_heads = config.num_attention_heads
        assert config.hidden_size % config.num_attention_heads == 0
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = tf.keras.layers.Dense(
            self.all_head_size, kernel_initializer=get_initializer(config.initializer_range)
        )
        self.key = tf.keras.layers.Dense(
            self.all_head_size, kernel_initializer=get_initializer(config.initializer_range)
        )
        self.value = tf.keras.layers.Dense(
            self.all_head_size, kernel_initializer=get_initializer(config.initializer_range)
        )

        self.dropout = tf.keras.layers.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_attention_heads, self.attention_head_size))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs, training=False):
        hidden_states, attention_mask, head_mask, output_attentions = inputs

        batch_size = shape_list(hidden_states)[0]
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer, batch_size)
        key_layer = self.transpose_for_scores(mixed_key_layer, batch_size)
        value_layer = self.transpose_for_scores(mixed_value_layer, batch_size)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = tf.matmul(
            query_layer, key_layer, transpose_b=True
        )  # (batch size, num_heads, seq_len_q, seq_len_k)
        dk = tf.cast(shape_list(key_layer)[-1], tf.float32)  # scale attention_scores
        attention_scores = attention_scores / tf.math.sqrt(dk)

        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in TFBertModel call() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = tf.nn.softmax(attention_scores, axis=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs, training=training)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = tf.matmul(attention_probs, value_layer)

        context_layer = tf.transpose(context_layer, perm=[0, 2, 1, 3])
        context_layer = tf.reshape(
            context_layer, (batch_size, -1, self.all_head_size)
        )  # (batch_size, seq_len_q, all_head_size)

        outputs = (
            (context_layer, attention_probs) if cast_bool_to_primitive(output_attentions) is True else (context_layer,)
        )

        return outputs

In [ ]:
def MyTFBertAttention(inputs, config, training=True):
    input_tensor, attention_mask, head_mask, output_attentions = inputs
    self_attention = TFBertSelfAttention(config)
    dense_output = TFBertSelfOutput(config)
    
    self_outputs = self_attention(
        [input_tensor, attention_mask, head_mask, output_attentions], training=training
    )
    attention_output = dense_output([self_outputs[0], input_tensor], training=training)
    outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them
    return outputs

In [ ]:
def MyTFBertLayer(inputs, config, training=True):
    hidden_states, attention_mask, head_mask, output_attentions = inputs
    
    intermediate = TFBertIntermediate(config)
    bert_output = TFBertOutput(config)
    
    attention_outputs = MyTFBertAttention([hidden_states, attention_mask, head_mask, output_attentions], config, training=training)
    attention_output = attention_outputs[0]
    
    intermediate_output = intermediate(attention_output)
    layer_output = bert_output([intermediate_output, attention_output], training=training)
    outputs = (layer_output,) + attention_outputs[1:]  # add attentions if we output them
    return outputs

In [ ]:
def MyTFBertEncoder(inputs, config, training=True):
    hidden_states, attention_mask, head_mask, output_attentions = inputs
    output_hidden_states = config.output_hidden_states
    all_hidden_states = ()
    all_attentions = ()
    
    for i in range(config.num_hidden_layers):
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        layer_outputs = MyTFBertLayer([hidden_states, attention_mask, head_mask[i], output_attentions], config, training=training)
        hidden_states = layer_outputs[0]

        if cast_bool_to_primitive(output_attentions) is True:
            all_attentions = all_attentions + (layer_outputs[1],)

    # Add last layer
    if output_hidden_states:
        all_hidden_states = all_hidden_states + (hidden_states,)

    outputs = (hidden_states,)
    if output_hidden_states:
        outputs = outputs + (all_hidden_states,)
    if cast_bool_to_primitive(output_attentions) is True:
        outputs = outputs + (all_attentions,)
    return outputs  # outputs, (hidden states), (attentions)

In [ ]:
def MyBertModel(pre_embeds):
    config = BertConfig(
        intermediate_size=128,
        max_position_embeddings=maxlen,
        num_attention_heads=4,
        num_hidden_layers=2,
        type_vocab_size=1,
        vocab_size=len(pre_embeds),
        hidden_size=embed_size
    )
    retbyemb = MyTFBertEmbedding(pre_embeds, config)
    
    inps = retbyemb["inputs"]
    embeds = retbyemb["embeddings"]
    
    inp_atten_mask = layers.Input(shape=(1, 1, maxlen))
    head_mask = [None] * config.num_hidden_layers
    inputs = embeds, inp_atten_mask, head_mask, config.output_attentions
    outputs = MyTFBertEncoder(inputs, config)
    
    ret = {"inputs": inps+[inp_atten_mask],
           "layers": outputs
          }
    
    return ret

In [ ]:
ret_1 = MyBertModel(emb_matrix_dic[id1])
inps_1 = ret_1["inputs"]
bert_layer_1 = ret_1["layers"]

ret_2 = MyBertModel(emb_matrix_dic[id2])
inps_2 = ret_2["inputs"]
bert_layer_2 = ret_2["layers"]

ret_3 = MyBertModel(emb_matrix_dic[id3])
inps_3 = ret_3["inputs"]
bert_layer_3 = ret_3["layers"]

# ret_4 = MyBertModel(emb_matrix_dic[id4])
# inps_4 = ret_4["inputs"]
# bert_layer_4 = ret_4["layers"]

# ret_5 = MyBertModel(emb_matrix_dic[id5])
# inps_5 = ret_5["inputs"]
# bert_layer_5 = ret_5["layers"]

In [ ]:
bilstm_1 = layers.LSTM(128, return_sequences=True)(bert_layer_1[0])
bilstm_2 = layers.LSTM(128, return_sequences=True)(bert_layer_2[0])
bilstm_3 = layers.LSTM(128, return_sequences=True)(bert_layer_3[0])
# bilstm_4 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(bert_layer_4[0])
# bilstm_5 = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(bert_layer_5[0])

In [ ]:
maxpool_1 = layers.GlobalMaxPooling1D()(bilstm_1)
maxpool_2 = layers.GlobalMaxPooling1D()(bilstm_2)
maxpool_3 = layers.GlobalMaxPooling1D()(bilstm_3)
# maxpool_4 = layers.GlobalMaxPooling1D()(bilstm_4)
# maxpool_5 = layers.GlobalMaxPooling1D()(bilstm_5)

In [ ]:
conc_list = [maxpool_1, maxpool_2, maxpool_3,]

In [ ]:
conc = layers.concatenate(conc_list)

In [ ]:
conc = keras.layers.Dense(256)(conc)
conc = keras.layers.BatchNormalization()(conc)
conc = keras.layers.PReLU()(conc)
conc = keras.layers.Dropout(0.2)(conc)

conc = keras.layers.Dense(128)(conc)
conc = keras.layers.BatchNormalization()(conc)

out_age = keras.layers.Dense(10, activation="softmax", name="out_age")(conc)
out_gender = keras.layers.Dense(1, activation="sigmoid", name="out_gender")(conc)

model = keras.Model(inputs=[inps_1, inps_2, inps_3], outputs=[out_age, out_gender])


model.compile(
    loss={"out_age": "categorical_crossentropy", "out_gender": "binary_crossentropy"},
    loss_weights={"out_age": 1, "out_gender": 1},
    optimizer=tfa.optimizers.RectifiedAdam(lr=0.001), 
    metrics=["acc"]
)

model.summary()

In [ ]:
# ids, pos, token, mask
train_set = list()
test_set = list()
for col in TFBS:
    train_set += [train_textprec_dic[col], train_pos_ids[col], train_token_ids[col], train_mask.reshape(-1,1,1,maxlen)]
    test_set += [test_textprec_dic[col], test_pos_ids[col], test_token_ids[col], test_mask.reshape(-1,1,1,maxlen)]

In [ ]:
# train_set = [train_txtprec, train_txtprec, train_mask, train_mask.reshape(-1,1,1,maxlen)]
# test_set = [test_txtprec, test_txtprec, train_mask, test_mask.reshape(-1,1,1,maxlen)]

In [ ]:
train_y_age = keras.utils.to_categorical(train_user["age"]-1)
train_y_gender = (train_user["gender"]-1).values.reshape(-1, 1)

In [ ]:
res_set = train_test_split(*train_set, train_y_age, train_y_gender, test_size=0.2, random_state=2020)

In [ ]:
train_X_tr = res_set[:-4:2]
train_X_val = res_set[1:-4:2]
train_y_tr_age = res_set[-4]
train_y_val_age = res_set[-3]

train_y_tr_gender = res_set[-2]
train_y_val_gender = res_set[-1]

In [ ]:
gc.collect()

In [ ]:
elsp = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0)
filepath = f"{UMDIR}/weights_improvement_fold.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only=True, mode="min")

In [ ]:
cb = [elsp,]  # checkpoint]

In [ ]:
model.fit(train_X_tr, [train_y_tr_age, train_y_tr_gender], validation_data=(train_X_val, [train_y_val_age, train_y_val_gender]), epochs=3, callbacks=cb)

In [ ]:
# model = keras.models.load_model(f"{UMDIR}/3inp_bert_4heads2layers_model_emb{embed_size}_seq{maxlen}.h5")
# model = keras.models.load_model(f"{UMDIR}/5inp_2bilstm_model_128.h5")
model._layers = [
    layer for layer in model._layers if isinstance(layer, keras.layers.Layer)
]
keras.utils.plot_model(model, to_file=f"{UMDIR}/3inp_bert_4heads2layers_model_emb{embed_size}_seq{maxlen}.png", show_layer_names=False)

In [ ]:
1411/60

## Keras Transfomers

In [ ]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
# Position encoding
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # 将 sin 应用于数组中的偶数索引（indices）；2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # 将 cos 应用于数组中的奇数索引；2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, emded_dim, embedding_matrix):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=len(embedding_matrix), output_dim=emded_dim, weights=[embedding_matrix], trainable=False)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim, weights=positional_encoding(maxlen, embed_size), trainable=False)
        self.emded_dim = emded_dim
        self.maxlen = maxlen

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
num_heads = 4  # Number of attention heads
ff_dim = 128

num_enc = 2

In [ ]:
id1 = "creative_id"
id2 = "ad_id"
id3 = "product_id"

# id1 = "creative_id"
# id2 = "ad_id"
# id3 = "advertiser_id"
# id4 = "product_id"
# id5 = "industry"
# # id6 = "product_category"
# TFBS = [id1, id2, id4]

In [ ]:
inp_1 = layers.Input(shape=(maxlen,))
inp_2 = layers.Input(shape=(maxlen,))
inp_3 = layers.Input(shape=(maxlen,))

In [ ]:
embed_1 = TokenAndPositionEmbedding(maxlen, embed_size, emb_matrix_dic[id1])(inp_1)
embed_2 = TokenAndPositionEmbedding(maxlen, embed_size, emb_matrix_dic[id1])(inp_2)
embed_3 = TokenAndPositionEmbedding(maxlen, embed_size, emb_matrix_dic[id1])(inp_3)

In [ ]:
transf_1 = list()
transf_2 = list()
transf_3 = list()
for i in range(num_enc):
    if i == 0:
        transf_1.append(TransformerBlock(embed_size, num_heads, ff_dim)(embed_1))
        transf_2.append(TransformerBlock(embed_size, num_heads, ff_dim)(embed_2))
        transf_3.append(TransformerBlock(embed_size, num_heads, ff_dim)(embed_3))
    else:
        transf_1.append(TransformerBlock(embed_size, num_heads, ff_dim)(transf_1[-1]))
        transf_2.append(TransformerBlock(embed_size, num_heads, ff_dim)(transf_2[-1]))
        transf_3.append(TransformerBlock(embed_size, num_heads, ff_dim)(transf_3[-1]))

In [ ]:
# bilstm_1 = list()
# bilstm_2 = list()
# bilstm_3 = list()
# for i in range(num_enc):
#     bilstm_1.append(layers.LSTM(128, return_sequences=True)(transf_1[i]))
#     bilstm_2.append(layers.LSTM(128, return_sequences=True)(transf_2[i]))
#     bilstm_3.append(layers.LSTM(128, return_sequences=True)(transf_3[i]))

In [ ]:
avgpool_1 = list()
avgpool_2 = list()
avgpool_3 = list()
for i in range(num_enc):
    avgpool_1.append(layers.GlobalMaxPooling1D()(transf_1[i]))
    avgpool_2.append(layers.GlobalMaxPooling1D()(transf_2[i]))
    avgpool_3.append(layers.GlobalMaxPooling1D()(transf_3[i]))

In [ ]:
# transf_1 = TransformerBlock(embed_size, num_heads, ff_dim)(embed_1)
# transf_2 = TransformerBlock(embed_size, num_heads, ff_dim)(embed_2)
# transf_3 = TransformerBlock(embed_size, num_heads, ff_dim)(embed_3)

In [ ]:
# avgpool_1 = layers.GlobalAveragePooling1D()(transf_1)
# avgpool_2 = layers.GlobalAveragePooling1D()(transf_2)
# avgpool_3 = layers.GlobalAveragePooling1D()(transf_3)

In [ ]:
# conc = layers.concatenate([avgpool_1, avgpool_2, avgpool_3])
conc = layers.concatenate(avgpool_1+avgpool_2+avgpool_3)

conc = layers.Dense(256)(conc)
conc = layers.BatchNormalization()(conc)
conc = layers.PReLU()(conc)
conc = layers.Dropout(0.2)(conc)

conc = layers.Dense(128)(conc)
conc = layers.BatchNormalization()(conc)

out_age = layers.Dense(10, activation="softmax", name="out_age")(conc)
out_gender = layers.Dense(1, activation="sigmoid", name="out_gender")(conc)

model = keras.Model(inputs=[inp_1, inp_2, inp_3,], outputs=[out_age, out_gender])

In [ ]:
model.compile(
    loss={"out_age": "categorical_crossentropy", "out_gender": "binary_crossentropy"},
    loss_weights={"out_age": 1, "out_gender": 1},
    optimizer=tfa.optimizers.RectifiedAdam(learning_rate=0.01), 
    metrics=["acc"]
)

In [ ]:
model.summary()

In [ ]:
train_set = [train_textprec_dic[id1], train_textprec_dic[id2], train_textprec_dic[id3]]
test_set = [test_textprec_dic[id1], test_textprec_dic[id2], test_textprec_dic[id3]]

In [ ]:
train_y_age = keras.utils.to_categorical(train_user["age"]-1)
train_y_gender = (train_user["gender"]-1).values.reshape(-1, 1)

In [ ]:
res_set = train_test_split(*train_set, train_y_age, train_y_gender, test_size=0.2, random_state=2020)

In [ ]:
train_X_tr = res_set[:-4:2]
train_X_val = res_set[1:-4:2]
train_y_tr_age = res_set[-4]
train_y_val_age = res_set[-3]

train_y_tr_gender = res_set[-2]
train_y_val_gender = res_set[-1]

In [ ]:
elsp = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0)
# filepath = f"{UMDIR}/weights_improvement_fold.hdf5"
# checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only=True, mode="min")

In [ ]:
cb = [elsp] #, checkpoint]

In [ ]:
model.fit(train_X_tr, [train_y_tr_age, train_y_tr_gender], validation_data=(train_X_val, [train_y_val_age, train_y_val_gender]), epochs=5, callbacks=cb)

In [ ]:
# model = keras.models.load_model(f"{UMDIR}/3inp_2bilstm_model_emb{embed_size}_seq{maxlen}.h5")
# model = keras.models.load_model(f"{UMDIR}/5inp_2bilstm_model_128.h5")
model._layers = [
    layer for layer in model._layers if isinstance(layer, keras.layers.Layer)
]
keras.utils.plot_model(model, to_file=f"{UMDIR}/3inp_transformer_4heads2layers_model_emb{embed_size}_seq{maxlen}.png", show_layer_names=False)

In [ ]:
# # Search weights

# class_num=10

# weights = [1.0]*class_num



# def search_weight(valid_y, raw_prob, init_weight=[1.0]*class_num, step=0.001):

#     weight = init_weight.copy()

#     f_best = accuracy_score(y_true=valid_y, y_pred=raw_prob.argmax(

#         axis=1))

#     flag_score = 0

#     round_num = 1

#     while(flag_score != f_best):

#         print("round: ", round_num)

#         round_num += 1

#         flag_score = f_best

#         for c in range(class_num):

#             for n_w in range(0, 2000,10):

#                 num = n_w * step

#                 new_weight = weight.copy()

#                 new_weight[c] = num



#                 prob_df = raw_prob.copy()

#                 prob_df = prob_df * np.array(new_weight)



#                 f = accuracy_score(y_true=valid_y, y_pred=prob_df.argmax(

#                     axis=1))

#                 if f > f_best:

#                     weight = new_weight.copy()

#                     f_best = f

#                     print(f)

#     return weight

# Generate Prediction Result

In [ ]:
res = test_user[[UID]]

In [ ]:
res["predicted_age"] = np.argmax(predicted_age, axis=1)+1

In [ ]:
res["predicted_gender"] = (predicted_gender >= 0.5).astype(int)+1

In [ ]:
res_suffix = time.strftime("%Y%m%d%H%M%S", time.localtime(int(time.time())))
res.to_csv(f"{RESDIR}/res-{res_suffix}.csv", index=False)

In [ ]:
res["predicted_age"].value_counts()

In [ ]:
res["predicted_gender"].value_counts()

In [ ]:
gc.collect()

# Cent result to COS

In [ ]:
from ti import session
ti_session = session.Session()

In [ ]:
inputs = ti_session.upload_data(path=f"{RESDIR}/res-20200515004850.csv", bucket="etveritas-1252104022", key_prefix=RESDIR)